In [59]:
from io import StringIO

from mccd.random_clifford_circuit import *
from surface_sim.setups import CircuitNoiseSetup
from surface_sim.models import CircuitNoiseModel
from surface_sim import Detectors
from surface_sim.experiments import schedule_from_circuit, experiment_from_schedule
import time
import stim

from pathlib import Path
import stim
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from joblib import Parallel, delayed
import itertools
import shelve

from pymatching import Matching as MWPM
from mle_decoder import MLEDecoder as MLE
from stimbposd import BPOSD
from sklearn.metrics import accuracy_score

DECODER_BASELINES = {
    'BPOSD': BPOSD,
    'MLE': MLE,
    'MWPM': MWPM,
}

In [18]:
from surface_sim.circuit_blocks.rot_surface_code_css import gate_to_iterator
print('Rotated', gate_to_iterator.keys())
from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
print('Unrotated', gate_to_iterator.keys())
ROT_GATES = list('IXZ')
UNR_GATES = list('IHXZ')
MCCD_GATES = ['I', 'X', 'Y', 'Z', 'H']

Rotated dict_keys(['TICK', 'I', 'S', 'X', 'Z', 'CX', 'CNOT', 'R', 'RZ', 'RX', 'M', 'MZ', 'MX'])
Unrotated dict_keys(['TICK', 'I', 'S', 'H', 'X', 'Z', 'CX', 'CNOT', 'R', 'RZ', 'RX', 'M', 'MZ', 'MX'])


In [61]:
def to_stim_circuit(mccd_circuit):
    res = stim.Circuit()
    # Must have R and M. Error inactive layout.
    for n in range(mccd_circuit.n_logical_qubits):
        res.append('R', [n])

    for name, timestep, qubits in mccd_circuit:
        res.append(name, qubits)

    for n in range(mccd_circuit.n_logical_qubits):
        res.append('M', [n])

    return res

def print_random_circuit(c: RandomCliffordCircuit):
    return list(c)

def dict_product(input_dict):
    keys = input_dict.keys()
    value_lists = input_dict.values()

    # 使用itertools.product生成所有值的组合
    value_combinations = itertools.product(*value_lists)

    # 将每个值的组合与键配对，生成字典列表
    for combo in value_combinations:
        yield dict(zip(keys, combo))

def run_decoder(name: str, circuit: stim.Circuit, shots: int):
    method = DECODER_BASELINES[name](circuit.detector_error_model())
    sampler = circuit.compile_detector_sampler()
    syndrome, labels = sampler.sample(shots=shots, separate_observables=True)
    begin = time.time_ns()
    predictions = method.decode_batch(syndrome)
    end = time.time_ns()
    logical_accuracy = accuracy_score(labels, predictions)
    walltime_seconds = (end - begin) / 1e9
    return dict(
        decoder=name,
        logical_accuracy=logical_accuracy,
        walltime_seconds=walltime_seconds,
    )

def compile_to_physical(log_cir: stim.Circuit, distance: int, noise_prob: float, rotated=True) -> stim.Circuit:
    if rotated:
        from surface_sim.circuit_blocks.rot_surface_code_css import gate_to_iterator
        from surface_sim.layouts import rot_surface_codes
        layouts = rot_surface_codes(log_cir.num_qubits, distance=distance)
    else:
        from surface_sim.circuit_blocks.unrot_surface_code_css import gate_to_iterator
        from surface_sim.layouts import unrot_surface_codes
        layouts = unrot_surface_codes(log_cir.num_qubits, distance=distance)

    setup = CircuitNoiseSetup()
    model = CircuitNoiseModel.from_layouts(setup, *layouts)
    detectors = Detectors.from_layouts("pre-gate", *layouts)
    setup.set_var_param("prob", noise_prob)
    schedule = schedule_from_circuit(log_cir, layouts, gate_to_iterator)
    phy_cir: stim.Circuit = experiment_from_schedule(
        schedule, model, detectors, anc_reset=True
    )
    return phy_cir


def random_mirror_symmetric_clifford(n_logical_qubits: int, circuit_index: str, depth: int, rotated=True) -> stim.Circuit:
    logical_class = TypeICircuit if circuit_index == '3' else TypeIICircuit
    random_circuit = logical_class(
        n_logical_qubits=n_logical_qubits,
        circuit_index=circuit_index,
        depth=depth,
        single_qubit_gate_list=ROT_GATES if rotated else ROT_GATES,
    )
    random_circuit.sample_circuit()
    return to_stim_circuit(random_circuit)



## Circuit Depths

Finally, to ensure that MCCD’s training generalizes to new circuits, we train MCCD with data from logical circuit depth D = 2, 4, 6, 8 and 10 (D = 4, 8, 12, 16 and 20) and test the model on a wide range of unseen circuits with depth up to D = 18 (D = 36) for Circuit Type I (Circuit Type II), defined below.

In [26]:
n_logical_qubits=2
circuit_index = '3'
depth = 2
code_distance=3 # 2 is not ok with rot code.
noise_prob = 1e-3
num_shots = 1000
num_circuits = 100


In [8]:
logical_class = TypeICircuit if circuit_index == '3' else TypeIICircuit

random_circuit = logical_class(
    n_logical_qubits=n_logical_qubits,
    circuit_index=circuit_index,
    depth=depth,
    single_qubit_gate_list=ROT_GATES)

random_circuit.sample_circuit()

In [9]:
random_circuit.to_string_lines()

['ZZ', 'ZI']

In [10]:
print_random_circuit(random_circuit)

[(np.str_('Z'), 0, [0]),
 (np.str_('Z'), 0, [1]),
 (np.str_('Z'), 1, [0]),
 (np.str_('I'), 1, [1]),
 (np.str_('Z'), 1, [0]),
 (np.str_('I'), 1, [1]),
 (np.str_('Z'), 0, [0]),
 (np.str_('Z'), 0, [1])]

In [11]:
random_circuit.n_logical_qubits

2

In [13]:
log_cir = to_stim_circuit(random_circuit)

In [67]:
print(log_cir)

R 0 1
I 0
Z 1
I 0 1 0 1 0
Z 1
M 0 1


In [14]:
from surface_sim.layouts import rot_surface_codes

layouts = rot_surface_codes(log_cir.num_qubits, distance=code_distance)
setup = CircuitNoiseSetup()
model = CircuitNoiseModel.from_layouts(setup, *layouts)
detectors = Detectors.from_layouts("pre-gate", *layouts)

setup.set_var_param("prob", 1e-3)

schedule = schedule_from_circuit(log_cir, layouts, gate_to_iterator)
phy_cir: stim.Circuit = experiment_from_schedule(
    schedule, model, detectors, anc_reset=True
)

In [15]:
print(phy_cir)

QUBIT_COORDS(0, 0) 0
QUBIT_COORDS(0, 2) 1
QUBIT_COORDS(0, 4) 2
QUBIT_COORDS(2, 0) 3
QUBIT_COORDS(2, 2) 4
QUBIT_COORDS(2, 4) 5
QUBIT_COORDS(4, 0) 6
QUBIT_COORDS(4, 2) 7
QUBIT_COORDS(4, 4) 8
QUBIT_COORDS(-1, 1) 9
QUBIT_COORDS(1, 3) 10
QUBIT_COORDS(3, 1) 11
QUBIT_COORDS(5, 3) 12
QUBIT_COORDS(1, 1) 13
QUBIT_COORDS(1, 5) 14
QUBIT_COORDS(3, -1) 15
QUBIT_COORDS(3, 3) 16
QUBIT_COORDS(0, 8) 17
QUBIT_COORDS(0, 10) 18
QUBIT_COORDS(0, 12) 19
QUBIT_COORDS(2, 8) 20
QUBIT_COORDS(2, 10) 21
QUBIT_COORDS(2, 12) 22
QUBIT_COORDS(4, 8) 23
QUBIT_COORDS(4, 10) 24
QUBIT_COORDS(4, 12) 25
QUBIT_COORDS(-1, 9) 26
QUBIT_COORDS(1, 11) 27
QUBIT_COORDS(3, 9) 28
QUBIT_COORDS(5, 11) 29
QUBIT_COORDS(1, 9) 30
QUBIT_COORDS(1, 13) 31
QUBIT_COORDS(3, 7) 32
QUBIT_COORDS(3, 11) 33
R 0 1 2 3 4 5 6 7 8 17 18 19 20 21 22 23 24 25
X_ERROR(0.001) 0 1 2 3 4 5 6 7 8 17 18 19 20 21 22 23 24 25
I 9 10 11 12 13 14 15 16 26 27 28 29 30 31 32 33
DEPOLARIZE1(0.001) 9 10 11 12 13 14 15 16 26 27 28 29 30 31 32 33
TICK
I 5 13 7 10 1 15 4 0 1

In [16]:
sampler = phy_cir.compile_detector_sampler()

In [23]:
syn, labels = sampler.sample(shots=10, separate_observables=True)

In [25]:
syn.shape

(10, 8)

In [27]:
syn.reshape((10, n_logical_qubits, depth, -1))

array([[[[False, False],
         [False, False]],

        [[False, False],
         [False, False]]],


       [[[False, False],
         [False, False]],

        [[False, False],
         [False,  True]]],


       [[[False, False],
         [False, False]],

        [[False, False],
         [False, False]]],


       [[[False, False],
         [False, False]],

        [[False, False],
         [False, False]]],


       [[[False, False],
         [False, False]],

        [[False, False],
         [False, False]]],


       [[[False, False],
         [False, False]],

        [[False, False],
         [False, False]]],


       [[[False, False],
         [False, False]],

        [[False, False],
         [False, False]]],


       [[[False, False],
         [False, False]],

        [[False, False],
         [False, False]]],


       [[[False, False],
         [False, False]],

        [[False, False],
         [False, False]]],


       [[[False, False],
         [False, Fals

In [22]:
n_logical_qubits=2
circuit_index = '4'
depth = 36
code_distance=3 # 2 is not ok with rot code.
noise_prob = 1e-3
num_shots = 1000
num_circuits = 10

for i in range(num_circuits):
    log_cir = random_mirror_symmetric_clifford(n_logical_qubits, circuit_index, depth)
    phy_cir = compile_to_physical(log_cir, code_distance, noise_prob)
    for name in DECODER_BASELINES.keys():
        res = run_decoder(name, phy_cir, shots=num_shots)
        print(res)


{'decoder': 'BPOSD', 'logical_accuracy': 0.997, 'walltime_seconds': 0.003882}
{'decoder': 'MLE', 'logical_accuracy': 0.999, 'walltime_seconds': 0.182786}
{'decoder': 'MWPM', 'logical_accuracy': 0.999, 'walltime_seconds': 9.3e-05}
{'decoder': 'BPOSD', 'logical_accuracy': 0.998, 'walltime_seconds': 0.004594}
{'decoder': 'MLE', 'logical_accuracy': 0.999, 'walltime_seconds': 0.19865}
{'decoder': 'MWPM', 'logical_accuracy': 0.999, 'walltime_seconds': 5.2e-05}
{'decoder': 'BPOSD', 'logical_accuracy': 0.998, 'walltime_seconds': 0.004971}
{'decoder': 'MLE', 'logical_accuracy': 1.0, 'walltime_seconds': 0.230359}
{'decoder': 'MWPM', 'logical_accuracy': 0.999, 'walltime_seconds': 5.4e-05}
{'decoder': 'BPOSD', 'logical_accuracy': 1.0, 'walltime_seconds': 0.004142}
{'decoder': 'MLE', 'logical_accuracy': 0.999, 'walltime_seconds': 0.179817}
{'decoder': 'MWPM', 'logical_accuracy': 0.998, 'walltime_seconds': 4.8e-05}
{'decoder': 'BPOSD', 'logical_accuracy': 0.999, 'walltime_seconds': 0.004808}
{'decod

## Reproduce Baselines

### Shots & Repeat
We evaluate each decoder over 20 independent runs. In each run, we randomly sample 1,000 syndrome trajectories from Type I circuits and average them to obtain a run-level performance estimate. We report the mean across the 20 runs, with error bars showing s.e.m.

TODO: Align the noise model.

In [50]:
num_shots = 1000
noise_prob = 1e-3
trial = 20

def get_baseline_settings():
    distance = [3, 5]
    depth = [4, 8, 12, 16, 20, 24, 28, 32, 36]
    # Type I one qubit. Type II two qubits.
    logical_index = ['3', '4']

    return dict_product(dict(
        distance=distance,
        depth=depth,
        logical_index=logical_index,
    ))

In [51]:
baselien_settings = list(get_baseline_settings())
len(baselien_settings)

36

In [57]:
def generate_circuit(distance: int, depth: int, logical_index: str):
    log_cir = random_mirror_symmetric_clifford(2 if logical_index == '4' else 1, circuit_index, depth)
    phy_cir: stim.Circuit = compile_to_physical(log_cir, distance, noise_prob)
    config = dict(distance=distance, depth=depth, logical_index=logical_index)
    return str(phy_cir), config

generate_circuit(distance=3, depth=4, logical_index='3')


('QUBIT_COORDS(0, 0) 0\nQUBIT_COORDS(0, 2) 1\nQUBIT_COORDS(0, 4) 2\nQUBIT_COORDS(2, 0) 3\nQUBIT_COORDS(2, 2) 4\nQUBIT_COORDS(2, 4) 5\nQUBIT_COORDS(4, 0) 6\nQUBIT_COORDS(4, 2) 7\nQUBIT_COORDS(4, 4) 8\nQUBIT_COORDS(-1, 1) 9\nQUBIT_COORDS(1, 3) 10\nQUBIT_COORDS(3, 1) 11\nQUBIT_COORDS(5, 3) 12\nQUBIT_COORDS(1, 1) 13\nQUBIT_COORDS(1, 5) 14\nQUBIT_COORDS(3, -1) 15\nQUBIT_COORDS(3, 3) 16\nR 0 1 2 3 4 5 6 7 8\nX_ERROR(0.001) 0 1 2 3 4 5 6 7 8\nI 9 10 11 12 13 14 15 16\nDEPOLARIZE1(0.001) 9 10 11 12 13 14 15 16\nTICK\nI 5 13 7 10 1 15 4 0 12 6 8 9 2 3 14 16 11\nDEPOLARIZE1(0.001) 5 13 7 10 1 15 4 0 12 6 8 9 2 3 14 16 11\nTICK\nTICK\nX 0 3 6\nDEPOLARIZE1(0.001) 0 3 6\nI 5 13 7 10 1 15 12 4 8 9 2 14 16 11\nDEPOLARIZE1(0.001) 5 13 7 10 1 15 12 4 8 9 2 14 16 11\nTICK\nTICK\nI 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16\nDEPOLARIZE1(0.001) 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16\nTICK\nTICK\nX 0 3 6\nDEPOLARIZE1(0.001) 0 3 6\nI 5 13 7 10 1 15 12 4 8 9 2 14 16 11\nDEPOLARIZE1(0.001) 5 13 7 10 1 15 12 4

In [58]:
def generate_circuit_tasks():
    for params in baselien_settings:
        yield delayed(generate_circuit)(**params)

bench_circuits = Parallel(n_jobs=-1, verbose=1)(generate_circuit_tasks())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not i

In [62]:
def run_decoder_tasks():
    def run_decoder_plus(config, cir_str, **kwargs):
        cir = stim.Circuit.from_file(StringIO(cir_str))
        kwargs['circuit'] = cir
        res = run_decoder(**kwargs)
        res.update(config)
        return res

    for cir, config in bench_circuits:
        for decoder in DECODER_BASELINES.keys():
            for t in range(trial):
                yield delayed(run_decoder_plus)(config, cir, name=decoder, shots=num_shots)

bench_result = Parallel(n_jobs=-1, verbose=1)(run_decoder_tasks())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.


Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29
Restricted license - for non-production use only - expires 2027-11-29


[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 395 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 773 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done 1260 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1790 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 2160 out of 2160 | elapsed:  5.8min finished


In [67]:
df = pd.DataFrame.from_records(bench_result)
df = pd.melt(df, id_vars=['decoder', 'distance', 'depth', 'logical_index'], value_vars=['walltime_seconds', 'logical_accuracy'], var_name='metric',
             value_name='value')

In [68]:
df.head()

,decoder,distance,depth,logical_index,metric,value
0,BPOSD,3,4,3,walltime_seconds,0.011079
1,BPOSD,3,4,3,walltime_seconds,0.003440
2,BPOSD,3,4,3,walltime_seconds,0.003515
3,BPOSD,3,4,3,walltime_seconds,0.003501
4,BPOSD,3,4,3,walltime_seconds,0.004473


In [69]:
df.to_csv('./data/bench_baselines.csv', index=False)


In [73]:
# Must save the circuits for fair comparison with MCCD.
# Filename format: d3_c3_D1, distance=3, circuit_index=3, depth=1
save_dir = Path('./data/bench/stim_circuits')
save_dir.mkdir(parents=True, exist_ok=True)

for cir_str, config in bench_circuits:
    distance, depth, logical_index = config['distance'], config['depth'], config['logical_index']
    filename = f'd{distance}_c{logical_index}_D{depth}.stim'
    filename = save_dir / filename
    filename.write_text(cir_str)
